# Tabulky s hierarchickým indexem

Tabulky s hierarchickým indexem umožňují v tabulce pracovat s daty, která mají složitější strukturu, než prosté dvourozměrné souhrny čísel. Například je možné pro různé scénáře (nastavení parametrů) řešit model s různými výchozími stavy (počátečními podmínkami) a vše zahrnout do jedné tabulky. Poté je možné například zpracovat všechny simulace s daným počátečním stavem (a libovolným nastavením dalších parametrů) nebo všechny simulace se s daným nastavením parametrů (a libovolnou počáteční podmínkou). 

Ukážeme si na rovnici logistického růstu s konstantním lovem.


Rovnice $$\frac{\mathrm dn}{\mathrm dt}=rn\left(1-\frac nK\right)-h$$
popisuje vývoj populace o velikosti $n$ v prostředí s nosnou kapacitou $K$. Budeme pro různou intenzitu lovu vždy sledovat řešení pro několik počátečních podmínek. Všechno zaznamenáme do tabulky a data z tabulky poté vykreslíme. Sloupec bude identifikován pomocí trojice hodnot, udávajících že jde o velikosti populací (a ne čas), dále intenzitu lovu a počáteční podmínku. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp


In [ ]:
pocatecni_podminka = [0.4]
meze = [0,10]

def rovnice(t, x, r=1, K=1, h=0):
    return r*x*(1-x/K) - h

def destrukce_populace(t,x,r=1,K=1,h=0):  # Pokud x klesne na nulu, zastavíme výpočet
    return x
destrukce_populace.terminal = True

t=np.linspace(*meze,101)  
seznam_h = [0,0.1,0.15,0.2,0.25,0.28]
seznam_pp = np.round(np.arange(0.1,1.2,0.05),2)

### Definice tabulky s víceúrovňovými nadpisy sloupců, MultiIndex
### https://pandas.pydata.org/docs/user_guide/advanced.html
iterables = [seznam_h,seznam_pp]
my_index = pd.MultiIndex.from_product(iterables, names=['lov', 'poč.podm.'])
df = pd.DataFrame(columns=my_index, dtype='float64', index=t)
df.index.name = "Čas"
df

Dále ve dvojitém cyklu pro každou počáteční podmínku a úroveň lovu najdeme
řešení. Hledání řešení ukončíme při dosažení konce časového intervalu, nebo při
dosažení nulové hodnoty pro velikost populace. Proto jsme tabulku založili
rovnou se sloupci, které mají nedefinované hodnoty
`np.nan`. Nyní vyměníme tolik položek ze začátku sloupce, kolik máme k
dispozici vypočtených hodnot. 

Tabulku vytiskneme, pro lepší přehlednost naležato. 

In [ ]:
for h in seznam_h:
    for pp in seznam_pp:
        reseni = solve_ivp(
                   lambda t,x:rovnice(t,x,h=h),
                   meze,
                   [pp],
                   t_eval=t, 
                   events=destrukce_populace,            
                   )
        df.loc[reseni.t,(h,pp)] = reseni.y.reshape(-1) # poté hodnoty zaměníme za vypočtené od začátku až po konec řešení
df.T   # vytiskneme tabulku naležato     

Na závěr tabulku vykreslíme. Například všechny křivky odpovídající stejné intenzitě lovu vykreslíme stejnou barvou.

In [ ]:
for i,h in enumerate(seznam_h):        
    plt.plot(df.index,df[h],color=f"C{i}",label=h,alpha=0.4,lw=1) 

ax = plt.gca()
ax.set(
    title = "Dynamika populace\n v závislosti na intenzitě lovu a počáteční podmínce",
    xlabel="Čas",
    ylabel="Velikost populace",
)
# Návod jak seskupit položky legendy je na https://stackoverflow.com/questions/26337493/pyplot-combine-multiple-line-labels-in-legend
handles, labels = ax.get_legend_handles_labels()
labels, ids = np.unique(labels, return_index=True)
handles = [handles[i] for i in ids]
ax.legend(handles, labels, title="Intenzita lovu");

Na závěr ukázka, jak snadno z tabulky vybereme všechna řešení s počáteční podmínkou 0.2. Jde vidět, že tři hodnoty lovu vedou k destrukci populace v konečném čase. Jako obvykle, je několik možností jak cíle dosáhnout, ukážeme si dvě z nich.

In [ ]:
df.loc[:,(slice(None),0.2)]

In [ ]:
df.xs(level="poč.podm.", key=0.2, axis=1)